In [ ]:
import os
import os.path as osp

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import numpy as np
import scipy.stats as ss

In [ ]:
#from google.colab import drive
#drive.mount('gdrive', force_remount=True)

In [ ]:
#DATA_PATH = r'gdrive/My Drive/Data'

#DATA_PATH = r'/scratch/gallowaa/cciw/Data'

DATA_PATH = osp.join(os.environ['DATA_PATH'], 'cciw/Data')

analysis_path = os.path.join(DATA_PATH, 'Tables', 'Analysis.csv')
dive_path = os.path.join(DATA_PATH, 'Tables', 'Dives.csv')

analysis_df = pd.read_csv(analysis_path, index_col=0,
                          dtype={'Count':float})
dive_df = pd.read_csv(dive_path, index_col=0, parse_dates=['Date'])

data_df = pd.merge(analysis_df, dive_df, on='Dive Index', how='outer')

data_df.columns

In [ ]:
X = data_df[['Silt (%)', 'Clay (%)', 'Sand (%)', 'Gravel (%)',
       'Cobble (%)', 'Rock (%)', 'Bedrock (%)', 'Boulders (%)', 'Shale (%)']]
X = pd.DataFrame(data=X.values/X.sum(axis=1).values.reshape(-1,1), columns=X.columns)
X

In [ ]:
#X.sum(axis=1)

In [ ]:
Y = data_df[['Live Coverage', 'Empty Coverage', 'Biomass',
       'Count', '16mm', '14mm', '12.5mm', '10mm', '8mm', '6.3mm', '4mm', '2mm']]
Y

### Cross-correlation

In [ ]:
X.corr()

In [ ]:
for column in Y.columns:
  y = Y[column]
  y_train = y.dropna()
  X_train = X.loc[y.notnull()]
  X_train['Mussels'] = y_train.values
  print(f"Cross-correlation for {column}:")
  print(X_train.corr()['Mussels'])
  print("")

### Least-square regression (a.k.a. multiple linear regression)

In [ ]:
def train(X, y):
  sol1 = np.linalg.lstsq(X, y) # Solve linear system (least-square solution)
  return sol1[0]

def predict(X, a):
  return X @ a.T

In [ ]:
for column in Y.columns:
  y = Y[column]
  y_train = y.dropna().values
  X_train = X.loc[y.notnull()].values
  a = train(X_train, y_train)
  y_pred = predict(X_train, a)
  r = ss.pearsonr(y_train, y_pred)[0]
  plt.plot(y_train, y_pred, '.')
  plt.plot()
  plt.title(f"Prediction of {column} from substrates. Linear correlation: {r}.")
  plt.xlabel("Observed")
  plt.xlabel("Predicted")
  plt.show()

### Hard vs soft substrate

In [ ]:
X.columns

In [ ]:
coverage_dict = {'Hard':X[['Gravel (%)', 'Cobble (%)',
       'Rock (%)', 'Bedrock (%)', 'Boulders (%)']].sum(axis=1).values,
 'Soft':X[['Silt (%)', 'Clay (%)', 'Sand (%)', 'Shale (%)']].sum(axis=1).values}
Xp = pd.DataFrame(data=coverage_dict)
Xp

In [ ]:
Xp['Mussels'] = Y['Live Coverage'].values

In [ ]:
Xp.corr()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(Xp['Hard'].values, Xp['Mussels'].values, '.')

In [ ]:
Xp['Mussels'].groupby(pd.cut(Xp["Hard"], np.arange(-0.05,1.15,0.1))).mean()

### By month

In [ ]:
X1 = X.loc[Y['Biomass'].notnull()]
X1

In [ ]:
X1['Month'] = data_df.loc[Y['Biomass'].notnull(),'Date'].dt.month
X1['Mussels'] = Y['Biomass'].dropna()

In [ ]:
X1.boxplot(by='Month', column='Mussels')

Largest potential biomass (95-th percentile) in July, first increases from April to July, then decreases until October

# Count vs Biomass

In [ ]:
x = Y['Count']
y = Y['Biomass']
valid = np.logical_and(x>0, y>0)
y1 = y[valid].values.reshape(-1,1)
x1 = x[valid].values.reshape(-1,1)
x0 = np.ones(x1.shape)

In [ ]:
y = np.log(y1)
x = np.log(x1)

In [ ]:
X = np.array([x0,x])
X = X.T.reshape(-1,2)

In [ ]:
sol,res,r,s = np.linalg.lstsq(X, y)
a = np.exp(sol[0])
b = sol[1]
print(f"Linear regression coefficients (ax**b): a:{a[0]:0.3}, b:{b[0]:0.3}")

In [ ]:
std = np.sqrt(res[0]/len(x))
print(f"Multiplicative error factor: [{np.exp(-1.96*std):3.3},{np.exp(1.96*std):3.3}]")

In [ ]:
fig = plt.figure()
ax = plt.gca()
Y.plot(ax=ax, x='Count', y='Biomass', style='.', loglog=True)
x_ = np.logspace(-1, 4, 100)
ax.plot(x_, a*x_**b, '-')
ax.plot(x_, np.exp(1.96*std)*a*x_**b, '--', color='k', alpha=0.5)
ax.plot(x_, np.exp(-1.96*std)*a*x_**b, '--', color='k', alpha=0.5)

In [ ]:
fig = plt.figure()
ax = plt.gca()
Y.plot(ax=ax, x='Count', y='Biomass', style='.')
x_ = np.logspace(-1, 3.6, 100)
ax.plot(x_, a*x_**b, '-')
ax.plot(x_, np.exp(1.96*std)*a*x_**b, '--', color='k', alpha=0.5)
ax.plot(x_, np.exp(-1.96*std)*a*x_**b, '--', color='k', alpha=0.5)